In [1]:
import ibapi
import ib_insync
import nest_asyncio
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract 
from ibapi.order import *
import time
import threading
from datetime import datetime
nest_asyncio.apply()
from ib_insync import Stock 
from ib_insync import MarketOrder
from datetime import datetime

In [2]:
ib = ib_insync.IB()
def onError(reqId, errorCode, errorString, advancedOrderRejectInfo=None):
    print(f"Error. Id: {reqId}, Code: {errorCode}, Msg: {errorString}")
def onOrderStatus(order):
    print(f"Order Status. order {order.orderId}, is {order.status}")

ib.errorEvent += onError
ib.orderStatusEvent += onOrderStatus
try:
    ib.connect("127.0.0.1", 7497,1)
    print("Connected to IB")
except Exception as e:
    print(f"Error connecting to IB: {e}")

Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:cafarm
Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:hfarm
Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:eufarmnj
Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:cashfarm
Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:usfuture
Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:afarm
Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:jfarm
Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:usfarm.nj
Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:usfarm
Error. Id: -1, Code: 2106, Msg: HMDS data farm connection is OK:euhmds
Error. Id: -1, Code: 2106, Msg: HMDS data farm connection is OK:fundfarm
Error. Id: -1, Code: 2106, Msg: HMDS data farm connection is OK:ushmds
Error. Id: -1, Code: 2158, Msg: Sec-def data farm connection is OK:secdefil
Connected to IB


In [3]:
try:
    server_time = ib.reqCurrentTime()
    print("Server time:", server_time)
except Exception as e:
    print(f"Error requesting server time: {e}")

Server time: 2025-07-05 19:47:22+00:00


In [7]:
local_time = datetime.now()
print("Local Time:", local_time)

Local Time: 2025-07-05 14:48:21.870017


In [ ]:
spy_contract = Stock('SPY', 'SMART', 'USD')

spy_market_order = MarketOrder('BUY', 1)

print("[INFO] Placeing the market order...")
trade = ib.placeOrder(spy_contract, spy_market_order)

print("[INFO] Order placed. Waiting for order status...")
while not trade.isDone():
    ib.waitOnUpdate()
order_status = trade.orderStatus.status
print(f"[INFO] Order Status: {order_status}")
open_orders = ib.openOrders()
if trade.order in open_orders:
    print("[Check] Order is still open.")
else:
    print("[Check] Order is not open anymore.(Likely filled or cancelled)")

portfolio = ib.portfolio()
spy_position = next((p for p in portfolio if p.contract.symbol == 'SPY'), None)
if spy_position:
    print(f"[Success] SPY Position: {spy_position.position} shares at average cost {spy_position.averageCost}")
else:
    print("[Check] No SPY position found in portfolio.")